In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neox-20b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    with torch.no_grad():
        for _, param in model.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8650752 || all params: 10597552128 || trainable%: 0.08162971878329976


In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PyPDF2 import PdfReader

pdf_path = "ML Review3.pdf"

# Open the PDF file
with open(pdf_path, "rb") as file:
    # Create a PDF reader object
    reader = PdfReader(file)

    # Initialize an empty list to store the extracted text
    extracted_text = []

    # Iterate over each page in the PDF
    for page_number in range(len(reader.pages)):
        text = reader.pages[page_number].extract_text()

        # Add the extracted text to the list
        extracted_text.append(text)

In [ ]:
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

max_length = 512  # Set the desired maximum length

tokenized_data = tokenizer(extracted_text, truncation=True, padding=True, max_length=max_length)


In [ ]:
input_data = {
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
}


In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data["input_ids"])

    def __getitem__(self, idx):
        return {key: self.data[key][idx] for key in self.data}


In [ ]:
custom_data = CustomDataset(input_data)


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
#tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=custom_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=3,
        warmup_steps=2,
        max_steps=8,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 24>:24                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1530 in train                    │
│                                                                                                  │
│   1527 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1528 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1529 │   │   )                                                                                 │
│ ❱ 1530 │   │   return inner_training_loop(                                                       │
│   1531 │   │   │   args=args,                                                                    │
│   1532 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1533 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1794 in _inner_training_loop     │
│                                                                                                  │
│   1791 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1792 │   │   │   │                                                                             │
│   1793 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1794 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1795 │   │   │   │                                                                             │
│   1796 │   │   │   │   if (                                                                      │
│   1797 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2630 in training_step            │
│                                                                                                  │
│   2627 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2628 │   │                                                                                     │
│   2629 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2630 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2631 │   │                                                                                     │
│   2632 │   │   if self.args.n_gpu > 1:                                                           │
│   2633 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2655 in compute_loss             │
│                                                                                                  │
│   2652 │   │   │   labels = inputs.pop("labels")                                                 │
│   2653 │   │   else:                                                                             │
│   2654 │   │   │   labels = None                                                                 │
│ ❱ 2655 │   │   outputs = model(**inputs)                                                         │
│   2656 │   │   # Save past state if it exists              

In [ ]:
text = input()
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
torch.cuda.empty_cache()